# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
#imports

from matplotlib import pyplot as plt
from sklearn import datasets
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.environment import Environment 
from azureml.core.model import InferenceConfig 
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
import pandas as pd
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [5]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

experiment_name = 'Hyperdrive-ML'

experiment=Experiment(ws, experiment_name)
experiment

quick-starts-ws-232086
aml-quickstarts-232086
southcentralus
510b94ba-e453-4417-988b-fbdc37b55ca7


Name,Workspace,Report Page,Docs Page
Hyperdrive-ML,quick-starts-ws-232086,Link to Azure Machine Learning studio,Link to Documentation


#### Create or Attach an AmlCompute cluster

In [7]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..............................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


#### Dataset

In [6]:
dataset = Dataset.get_by_name(ws, name='mpg-pred')

#view the data
dataset.take(5).to_pandas_dataframe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year
0,18.0,8,307.0,130,3504,12.0,70
1,15.0,8,350.0,165,3693,11.5,70
2,18.0,8,318.0,150,3436,11.0,70
3,16.0,8,304.0,150,3433,12.0,70
4,17.0,8,302.0,140,3449,10.5,70


Enviroment Build

In [8]:

# Setup environment for your training run - Docker Enviroment with depandencies in .yml file
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

#Build the Enviroment and confirm build
build = sklearn_env.build(workspace=ws)
build.wait_for_completion(show_output=True)

Building a non-registered environment is not supported. Registering environment.


Image Build Status: Running

2023/04/27 05:19:54 Downloading source code...
2023/04/27 05:19:55 Finished downloading source code
2023/04/27 05:19:55 Creating Docker network: acb_default_network, driver: 'bridge'
2023/04/27 05:19:56 Successfully set up Docker network: acb_default_network
2023/04/27 05:19:56 Setting up Docker configuration...
2023/04/27 05:19:56 Successfully set up Docker configuration
2023/04/27 05:19:56 Logging in to registry: 404b104daac942dba4685e524c6d0c38.azurecr.io
2023/04/27 05:19:57 Successfully logged into 404b104daac942dba4685e524c6d0c38.azurecr.io
2023/04/27 05:19:57 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/04/27 05:19:57 Scanning for dependencies...
2023/04/27 05:19:58 Successfully scanned dependencies
2023/04/27 05:19:58 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB

Step 1/21 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230120.v1@

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [10]:

# Specify parameter sampler -- Randomly sampling various Random Foresr Hyperparameter  
ps = RandomParameterSampling(
    {
        '--n_estimators' : choice(200, 300, 400),
        '--max_depth': choice(10, 20, 30, 40, 50),
        '--bootstrap': choice(1,0)
    }
)

# Specify a Policy -- Bandit policy for early termination
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")


# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                      script='train.py',
                      compute_target=compute_target,
                      environment=sklearn_env
                      )

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_concurrent_runs=4,
                                     max_total_runs=16,                                     
                                    )

In [11]:
#TODO: Submit your experiment

hyperdrive_run = experiment.submit(hyperdrive_config) #use the experiment to submit the hyperdrive config



## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [12]:
# Monitor HyperDrive runs 
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [13]:
hyperdrive_run.wait_for_completion(show_output=True)

RunId: HD_f6e488eb-276e-4663-897b-bd857be1e5aa
Web View: https://ml.azure.com/runs/HD_f6e488eb-276e-4663-897b-bd857be1e5aa?wsid=/subscriptions/510b94ba-e453-4417-988b-fbdc37b55ca7/resourcegroups/aml-quickstarts-232086/workspaces/quick-starts-ws-232086&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-04-27T05:34:12.592925][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-27T05:34:13.0494314Z][SCHEDULER][INFO]Scheduling job, id='HD_f6e488eb-276e-4663-897b-bd857be1e5aa_0' 
[2023-04-27T05:34:13.2682041Z][SCHEDULER][INFO]Scheduling job, id='HD_f6e488eb-276e-4663-897b-bd857be1e5aa_1' 
[2023-04-27T05:34:13.3584909Z][SCHEDULER][INFO]Scheduling job, id='HD_f6e488eb-276e-4663-897b-bd857be1e5aa_2' 
[2023-04-27T05:34:13.330401][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-27T05:34:13.5294187Z][SCHEDULER][INFO]Scheduling job, id='HD_f6e488eb-276e-4663-897b-bd857be1e5

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_d03f1b18983e69234ac6a729b97dd989/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error:     user_identity = identity.login_with_device_code(scopes=scopes, **kwargs)\n  File \"/azureml-envs/azureml_d03f1b18983e69234ac6a729b97dd989/lib/python3.6/site-packages/azureml/_vendor/azure_cli_core/auth/identity.py\", line 133, in login_with_device_code\n    return check_result(result)\n  File \"/azureml-envs/azureml_d03f1b18983e69234ac6a729b97dd989/lib/python3.6/site-packages/azureml/_vendor/azure_cli_core/auth/util.py\", line 125, in check_result\n    aad_error_handler(result, **kwargs)\n  File \"/azureml-envs/azureml_d03f1b18983e69234ac6a729b97dd989/lib/python3.6/site-packages/azureml/_vendor/azure_cli_core/auth/util.py\", line 30, in aad_error_

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:

# Get your best run and save the model from that run.



#Best Run and metrics
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print("best run details :",best_run.get_details())
print("best run file names :",best_run.get_file_names())
print("best run metrics :",best_run.get_metrics())



print(best_run)


In [ ]:
#TODO: Save the best model
best_run.register_model(model_name = 'hyperdrive_best_run.pkl', model_path = 'outputs/')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
service_name = 'Hyperdrive-Web-service'

inference_config = InferenceConfig(entry_script='train.py', environment=sklearn_env)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

